# LangGraph implementation

In [3]:
from langchain_core.tools import tool
from langchain_openai.chat_models import ChatOpenAI

from langgraph.prebuilt import ToolNode
from langgraph.graph import StateGraph, MessagesState, START, END

from nlp.NLSPipeline import extract_nutritional_features
from models.dev.faiss_indexes import FlatIndex

from dotenv import load_dotenv
from typing import Dict

# Load env
load_dotenv()

# Load index
index = FlatIndex("../../recipe_embeddings.json")

/Users/Anaqi_Amir/opt/anaconda3/envs/FlavorFusion/lib/python3.11/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
@tool("recipe_recommender")
def recipe_recommender(q: str) -> Dict:
    """Takes in user input, extracts relevant features, and output recommended recipes from database"""

    print(f"\n Parsed user input: \n {q} \n")

    extracted_features = extract_nutritional_features(q)

    print(f"\n Extracted features: \n {extracted_features} \n")

    recs = index.recommend_recipes(
        user_ingredients=extracted_features.user_ingredients,
        allergens=extracted_features.allergens,
        calories=extracted_features.calories,
        total_fat=extracted_features.total_fat,
        protein=extracted_features.protein,
        saturated_fat=extracted_features.saturated_fat,
        carbs=extracted_features.carbs,
        sodium=extracted_features.sodium,
        sugar=extracted_features.sugar,
        top_n=10,
    )

    print(f"\n recs: \n {recs} \n")

    return {"messages": recs}


@tool("final_answer")
def final_answer(recommendations: Dict) -> str:
    """Formats the query into a final answer"""
    return recommendations


tools = [recipe_recommender, final_answer]
tool_node = ToolNode(tools)


def router(state: MessagesState):
    messages = state["messages"]
    last_message = messages[-1]
    if last_message.tool_calls:
        return "recommender"
    return END


def chatbot(state: MessagesState):
    messages = state["messages"]
    response = llm.invoke(messages)
    return {"messages": [response]}


llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.25, max_tokens=500).bind_tools(
    tools
)

output = tool_node.invoke(
    {
        "messages": [
            llm.invoke(
                "Recommend me a recipe with around 500 calories, low fat (no more than 10g total fat), high protein, \n"
                "and please avoid macadamia nuts and whey protein. I love tomatoes and basil, but I don't like onions."
            )
        ]
    }
)

print(output)


 Parsed user input: 
 500 calories, low fat, high protein, no macadamia nuts, no whey protein, with tomatoes and basil, without onions 


 Extracted features: 
 user_ingredients=['tomatoes', 'basil', 'onions'] allergens=['macadamia nuts', 'whey protein'] calories=(500.0, 500.0) total_fat=(None, None) saturated_fat=(None, None) carbs=(None, None) sugar=(None, None) sodium=(None, None) protein=(None, None) 


 recs: 
 ['molasses crinkles', 'molasses crinkles', 'molasses crinkles', 'molasses crinkles', 'molasses crinkles', 'molasses crinkles', 'molasses crinkles', 'molasses crinkles', 'molasses crinkles', 'molasses crinkles'] 

{'messages': [ToolMessage(content='{"messages": ["molasses crinkles", "molasses crinkles", "molasses crinkles", "molasses crinkles", "molasses crinkles", "molasses crinkles", "molasses crinkles", "molasses crinkles", "molasses crinkles", "molasses crinkles"]}', name='recipe_recommender', tool_call_id='call_UCa9CurnPg9DHCrsfmVCLsAM')]}


## Graph Implementation

In [ ]:
# graph = StateGraph(MessagesState)

# graph.add_node("chatbot", chatbot)
# graph.add_node("recommender", recipe_recommender)
# graph.add_node("final_answer", final_answer)

# graph.add_edge(START, "chatbot")
# # graph.add_conditional_edges("chatbot", router, ["recommender", END])
# graph.add_edge("chatbot", "recommender")
# graph.add_edge("recommender", "final_answer")
# graph.add_edge("final_answer", END)

# app = graph.compile()

# if __name__ == "__main__":
#     tool_node.invoke({
#     "messages": [llm.invoke("Recommend me a recipe with around 500 calories, low fat (no more than 10g total fat), high protein, \n"
#     "and please avoid macadamia nuts and whey protein. I love tomatoes and basil, but I don't like onions.")]
# })